In [145]:
import numpy as np
import tensorflow as tf
import emoji
import pandas as pd
import json
import pickle


from keras_preprocessing.text import Tokenizer
from keras.models import load_model, model_from_json
from keras_preprocessing.sequence import pad_sequences

np.random.seed(0)


In [146]:
model_folder = "models/training set v.3.0"

model_name = "epochs_100_layers_55_50_45_40_35_30_25_20_10"
# model_name = "epochs_15_layers_32_64_128_256"

model_json_path = f"{model_folder}/json/{model_name}.json"

model_weight_path = f"{model_folder}/weights/{model_name}.h5"


Load the model

In [147]:

json_file = open(f"{model_json_path}", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights(model_weight_path)
print("Loaded model from disk")

Loaded model from disk


Function to get emoji from class

In [148]:
emoji_dictionary = {
    0: ":red_heart:",  # Love # X
    1: ":face_with_tears_of_joy:",  # Laughter X
    2: ":grinning_face_with_big_eyes:", # Happiness # X
    3: ":loudly_crying_face:",  # Sadness # X
    4: ":smiling_face_with_heart-eyes:",  # Adoration X
    5: ":fire:",  # Excitement X
    6: ":thumbs_up:",  # Approval
    7: ":folded_hands:",  # Gratitude X
    8: ":angry_face:",  # Anger X
    9: ":thinking_face:",  # Contemplation X
}

In [149]:
def label_to_emoji(label):
    return emoji.emojize(emoji_dictionary[label])

Add own data

In [150]:
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [151]:
test = [
    "I am seeing what is happening",
    "Does loading and unloading even work?",
    "Naniii",
    "Wtf is happening",
    "Im dying",
    "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA",
    "I like your hoodie",
    "Thank you for the help",
    "Your jokes are so funny",
    "I made it",
    "You are so hot",
    "I love you so much",
    "Good job!",
    "I am trying", "I want to cry", "This is just sad",
    "I love sleeeping :)",
    "Wait what is the syllabus even rip",
    "slayyy",
    "FIRE",
    "If you're happy and you know it clap your hands",
    "Less epochs is better huh?",
    "Confusion matrices make it less confusing",
    "Zero errors on first run are scary",
    "THIS FINALLY WORKS MWAHAHAHA",
    "Hapi hapi hapi",
    "I love it when the code runs",
]


test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen=loaded_model.layers[0].input_shape[1], padding='post', truncating='post')

See Results

In [152]:
y_pred = loaded_model.predict(Xtest)
y_pred = np.argmax(y_pred, axis=1)
y_pred

1/1 [==============================] - 2s 2s/step


array([9, 9, 9, 8, 3, 9, 9, 7, 1, 2, 5, 0, 1, 3, 3, 3, 0, 7, 9, 5, 4, 7,
       2, 9, 9, 9, 0], dtype=int64)

In [153]:
for i in range(len(test)):
    print(test[i], label_to_emoji(y_pred[i]))

I am seeing what is happening 🤔
Does loading and unloading even work? 🤔
Naniii 🤔
Wtf is happening 😠
Im dying 😭
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA 🤔
I like your hoodie 🤔
Thank you for the help 🙏
Your jokes are so funny 😂
I made it 😃
You are so hot 🔥
I love you so much ❤️
Good job! 😂
I am trying 😭
I want to cry 😭
This is just sad 😭
I love sleeeping :) ❤️
Wait what is the syllabus even rip 🙏
slayyy 🤔
FIRE 🔥
If you're happy and you know it clap your hands 😍
Less epochs is better huh? 🙏
Confusion matrices make it less confusing 😃
Zero errors on first run are scary 🤔
THIS FINALLY WORKS MWAHAHAHA 🤔
Hapi hapi hapi 🤔
I love it when the code runs ❤️


In [154]:
# Calculate accuracy
# These values are subjective
test_labels = [9,3,8,8,3,3,4,7,1,2,5,0,6,3,3,3,0,7,5,5,2,7,9,3,5,2,0]

correct_predictions = np.sum(y_pred == test_labels)
total_predictions = len(test_labels)
accuracy = correct_predictions / total_predictions


# Calculate loss
# Assuming you have a loss function defined or you're using a standard loss function like categorical crossentropy
loss = 0.0
for i in range(len(test_labels)):
    true_label = test_labels[i]
    predicted_value = y_pred[i]  
    loss += (true_label - predicted_value) ** 2
loss /= len(test_labels)
print(f'Test loss (MSE): {loss}')
print(f'Test accuracy: {accuracy}')

Test loss (MSE): 10.851851851851851
Test accuracy: 0.5925925925925926


In [155]:
from termcolor import colored

data = [(test[i], label_to_emoji(y_pred[i]), label_to_emoji(test_labels[i])) for i in range(len(test))]

# Define headers
headers = ['Test Sentence', 'Predicted Emoji', 'True Emoji']

# Display the headers

print(colored(f"{headers[0]:<70} | {headers[1]:<1} | {headers[2]}", 'yellow'))
print(colored('-' * 71 + '|' + '-' * 17 + '|' + '-' * (len(headers[2])+1), 'yellow'))

# Display the data
for i in range(len(data)):
    if data[i][1] == data[i][2]:
        print(colored(f"{data[i][0]:<70} | {data[i][1]}\t\t | {data[i][2]}", 'green'))
    else:
        print(colored(f"{data[i][0]:<70} | {data[i][1]}\t\t | {data[i][2]}", 'red'))

    


Test Sentence                                                          | Predicted Emoji | True Emoji
-----------------------------------------------------------------------|-----------------|-----------
I am seeing what is happening                                          | 🤔		 | 🤔
Does loading and unloading even work?                                  | 🤔		 | 😭
Naniii                                                                 | 🤔		 | 😠
Wtf is happening                                                       | 😠		 | 😠
Im dying                                                               | 😭		 | 😭
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA        | 🤔		 | 😭
I like your hoodie                                                     | 🤔		 | 😍
Thank you for the help                                                 | 🙏		 | 🙏
Your jokes are so funny                                                | 😂		 | 😂
I made it                                                          